In [74]:
from datetime import datetime, timedelta
import time
import random
import matplotlib.pyplot as plt
import numpy as np
import logging
import os

In [ ]:
# Global state
failed_logins = {}
toggle_events = {}
power_history = []
session_creations = {}
device_access_log = {}
flagged_users = set()
flagged_ips = set()
session_activity = {}
event_counts = {'login_attempt': {}, 'toggle_device': {}, 'power_reading': {}, 'session_start': {}, 'session_end': {}}
login_locations = {}  # Tracks login info: (user_id, ip, timestamp, country)
session_durations = {}  # Tracks session info: {session_id: {start, end, last_active, type}}

In [ ]:
# Simulated IP-to-country mapping
ip_to_country = {
    '192.168.1.1': 'US', '192.168.1.10': 'US', '192.168.2.2': 'US', 
    '192.168.5.1': 'US', '192.168.5.2': 'US', '192.168.5.3': 'US', 
    '192.168.5.4': 'US', '192.168.6.1': 'US', '192.168.13.69': 'US', 
    '123.225.115.42': 'US', '22.15.1.3': 'US',
    '203.0.113.1': 'UK', '203.0.113.55': 'UK', '155.23.65.1': 'UK',
    '172.16.1.1': 'CN', '192.122.36.1': 'CN', '192.111.23.6': 'CN'
}

# Simulated travel times (hours) between countries
travel_times = {('US', 'UK'): 7, ('US', 'CN'): 14, ('UK', 'CN'): 10}

In [ ]:
detection_counter = 1

In [ ]:
def log_anomaly(user_id, source_id, message):
    global detection_counter
    now = datetime.now()
    timestamp_str = now.strftime('%Y-%m-%d, %H:%M:%S')
    log_line = f"{detection_counter}, {timestamp_str}, {user_id}, {source_id}, {message}\n"

    # Write to file
    with open("security_logs.txt", "a") as f:
        f.write(log_line)

    detection_counter += 1  # Increment for next use


if not os.path.exists("anomaly_logs.txt"):
    with open("anomaly_logs.txt", "w") as f:
        f.write("Detection, Date, Time, User, Source, Message\n")

In [77]:
def is_business_hours(timestamp):
    return 9 <= timestamp.hour <= 17

In [ ]:
def instrument(event_name, user_role, user_id, source_id, timestamp, context):
    print(f'Event: {event_name}, Role: {user_role}, User: {user_id}, Time: {timestamp}')

    # Update event counts for visualisation
    event_counts[event_name].setdefault(user_id, 0)
    event_counts[event_name][user_id] += 1

    # Update last active timestamp for session
    session_id = context.get('session_id')
    if session_id and session_id in session_durations:
        session_durations[session_id]['last_active'] = timestamp

    # Check for previously flagged user or IP
    if user_id in flagged_users or source_id in flagged_ips:
        alert_msg = f'Activity from previously flagged source: {user_id} / {source_id}'
        print(f"ALERT: {alert_msg}")
        log_anomaly(user_id, source_id, alert_msg)

    # Failed login detection
    if event_name == 'login_attempt':
        success = context.get('success', True)
        if not success:
            failed_logins.setdefault(user_id, []).append(timestamp)
            recent = [t for t in failed_logins[user_id] if (timestamp - t).total_seconds() <= 60]
            failed_logins[user_id] = recent
            if len(recent) > 5:
                flagged_users.add(user_id)
                flagged_ips.add(source_id)
                alert_msg = f'Too many failed login attempts for {user_id}'
                print(f"ALERT: {alert_msg}")
                log_anomaly(user_id, source_id, alert_msg)
                
        # Unusual login location detection
        country = ip_to_country.get(source_id, 'UNKNOWN')
        login_locations.setdefault(user_id, []).append((source_id, timestamp, country))
        recent_locations = [(ip, t, c) for ip, t, c in login_locations[user_id] if (timestamp - t).total_seconds() <= 300]
        login_locations[user_id] = recent_locations
        countries = set(c for _, _, c in recent_locations)

        if len(countries) > 2:
            # Check travel time feasibility
            for i, (_, t1, c1) in enumerate(recent_locations):
                for _, t2, c2 in recent_locations[i+1:]:
                    if c1 != c2:
                        hours = (t2 - t1).total_seconds() / 3600
                        min_travel = travel_times.get((min(c1, c2), max(c1, c2)), 5)
                        if hours < min_travel:
                            alert_msg = f'Unrealistic login locations for {user_id}: {c1} to {c2} in {hours:.2f} hours'
                            print(f"ALERT: {alert_msg}")
                            log_anomaly(user_id, source_id, alert_msg)
                            flagged_users.add(user_id)
                            flagged_ips.update(ip for ip, _, _ in recent_locations)
                            break

    # Toggle spam and device overload detection
    if event_name == 'toggle_device':
        toggle_events.setdefault(user_id, []).append(timestamp)
        recent = [t for t in toggle_events[user_id] if (timestamp - t).total_seconds() <= 30]
        toggle_events[user_id] = recent
        device_id = context.get('device_id', 'unknown')
        device_type = context.get('device_type', 'non-critical')
        toggle_state = context.get('state', 'on')
        device_access_log.setdefault(device_id, []).append((user_id, timestamp, toggle_state))
        if len(recent) > 10:
            if user_role not in ('ADMIN', 'MANAGER') or not is_business_hours(timestamp):
                alert_msg = f'Device toggled too frequently by {user_id}'
                print(f"ALERT: {alert_msg}")
                log_anomaly(user_id, source_id, alert_msg)
                flagged_users.add(user_id)
                flagged_ips.add(source_id)

        # Device overload detection
        recent_access = [(u, t, s) for u, t, s in device_access_log[device_id] if (timestamp - t).total_seconds() <= 10]
        device_access_log[device_id] = recent_access
        unique_users = set(u for u, _, _ in recent_access)
        threshold = 2 if device_type == 'critical' else 4
        if len(unique_users) > threshold:
            alert_msg = f'Device overload on {device_id} ({device_type}) by users: {unique_users}'
            print(f"ALERT: {alert_msg}")
            log_anomaly(user_id, source_id, alert_msg)
            flagged_users.update(unique_users)

        # Conflicting toggle state detection
        states = [s for _, _, s in recent_access]
        if len(states) >= 5 and states[-1] != states[-2]:
            alert_msg = f'Conflicting toggle states on {device_id}: {states[-2]} to {states[-1]}'
            print(f"ALERT: {alert_msg}")
            log_anomaly(user_id, source_id, alert_msg)
            flagged_users.update(unique_users)

    # Power anomaly detection
    if event_name == 'power_reading':
        value = context.get('value', 0)
        power_history.append((timestamp, value))

        if len(power_history) > 20:
            power_history.pop(0)
        avg = sum(v for _, v in power_history) / len(power_history)

        if value <= 0:
            alert_msg = f'Invalid power reading ({value}) from {source_id}'
            print(f"ALERT: {alert_msg}")
            log_anomaly(user_id, source_id, alert_msg)
            flagged_users.add(user_id)
            flagged_ips.add(source_id)

        elif value > 1.5 * avg:
            if user_role not in ('ADMIN', 'MANAGER') or not is_business_hours(timestamp):
                alert_msg = f'Power spike ({value}) exceeds 150% of avg ({avg:.2f})'
                print(f"ALERT: {alert_msg}")
                log_anomaly(user_id, source_id, alert_msg)
                flagged_users.add(user_id)
                flagged_ips.add(source_id)

    # Session hijacking and duration detection
    if event_name == 'session_start':
        session_id = context.get('session_id')
        ip = source_id
        session_type = context.get('session_type', 'interactive')
        if not session_id:
            return
        session_creations.setdefault(session_id, []).append((user_id, timestamp))
        session_activity.setdefault(session_id, []).append((ip, timestamp))
        session_durations[session_id] = {'start': timestamp, 'end': None, 'last_active': timestamp, 'type': session_type}
        recent = [(ip_old, ts_old) for ip_old, ts_old in session_activity[session_id] if (timestamp - ts_old).total_seconds() <= 60]
        session_activity[session_id] = recent
        unique_ips = set(ip_old for ip_old, _ in recent)
        if len(unique_ips) > 1:
            alert_msg = f'Session hijacking suspected for session \'{session_id}\' used from multiple IPs: {unique_ips}'
            print(f"ALERT: {alert_msg}")
            log_anomaly(user_id, source_id, alert_msg)

    if event_name == 'session_end':
        session_id = context.get('session_id')
        if not session_id or session_id not in session_durations:
            return
        session_durations[session_id]['end'] = timestamp
        duration = (timestamp - session_durations[session_id]['start']).total_seconds() / 3600
        idle_time = (timestamp - session_durations[session_id]['last_active']).total_seconds() / 3600
        threshold = 24 if session_durations[session_id]['type'] == 'interactive' else 48
        if duration > threshold and idle_time > 12:
            alert_msg = f'Excessive idle session duration for {session_id}({session_durations[session_id]}): {duration:.2f} hours, idle for {idle_time:.2f} hours'
            print(f"ALERT: {alert_msg}")
            log_anomaly(user_id, source_id, alert_msg)
            flagged_users.add(user_id)
            flagged_ips.add(source_id)

In [ ]:
def visualise_data():
    # Event frequency plot
    plt.figure(figsize=(10, 6))
    users = set().union(*[set(counts.keys()) for counts in event_counts.values()])
    x = np.arange(len(users))
    width = 0.2
    
    for i, event in enumerate(event_counts):
        counts = [event_counts[event].get(user, 0) for user in users]
        plt.bar(x + i * width, counts, width, label=event)
    
    plt.xlabel('Users')
    plt.ylabel('Event Count')
    plt.title('Event Frequency by User')
    plt.xticks(x + width * 2, users, rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig('event_frequency.png')
    plt.close()

    # Power readings plot
    if power_history:
        timestamps, values = zip(*power_history)
        plt.figure(figsize=(10, 6))
        plt.plot(timestamps, values, 'b-', label='Power Readings')
        avg = sum(values) / len(values)
        plt.axhline(y=avg, color='r', linestyle='--', label=f'Average ({avg:.2f})')
        plt.xlabel('Time')
        plt.ylabel('Power Value')
        plt.title('Power Readings Over Time')
        plt.legend()
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('power_readings.png')
        plt.close()

    # Session duration plot
    if session_durations:
        durations = []
        session_ids = []
        
        for sid, times in session_durations.items():
            if times['end']:
                duration = (times['end'] - times['start']).total_seconds() / 3600
                durations.append(duration)
                session_ids.append(sid)
        
        if durations:
            plt.figure(figsize=(10, 6))
            plt.bar(session_ids, durations, color='green')
            plt.axhline(y=24, color='r', linestyle='--', label='24-hour interactive threshold')
            plt.axhline(y=48, color='b', linestyle='--', label='48-hour background threshold')
            plt.xlabel('Session ID')
            plt.ylabel('Duration (hours)')
            plt.title('Session Durations')
            plt.legend()
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig('session_durations.png')
            plt.close()

# Normal User Simulation

In [80]:
def sim_normal_use(user_id, user_ip, timestamp=None):
    if timestamp is None:
        timestamp = datetime.now()
    instrument('login_attempt', 'USER', user_id, user_ip, timestamp, {'success': True})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('toggle_device', 'USER', user_id, user_ip, datetime.now(), {'device_id': 'device_1', 'device_type': 'non-critical', 'state': 'on'})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('power_reading', 'USER', user_id, user_ip, datetime.now(), {'value': 1000})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('session_start', 'USER', user_id, user_ip, datetime.now(), {'session_id': 'normal_session', 'session_type': 'interactive'})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('session_end', 'USER', user_id, user_ip, datetime.now(), {'session_id': 'normal_session'})

sim_normal_use('normal_user', '123.225.115.42')

Event: login_attempt, Role: USER, User: normal_user, Time: 2025-05-31 17:08:51.296643
Event: toggle_device, Role: USER, User: normal_user, Time: 2025-05-31 17:08:51.996071
Event: power_reading, Role: USER, User: normal_user, Time: 2025-05-31 17:08:53.434284
Event: session_start, Role: USER, User: normal_user, Time: 2025-05-31 17:08:54.300706
Event: session_end, Role: USER, User: normal_user, Time: 2025-05-31 17:08:55.585204


# Failed Login Simulator

In [81]:
def sim_failed_login(user_id, user_ip, count=6):
    for _ in range(count):
        instrument('login_attempt', 'USER', user_id, user_ip, datetime.now(), {'success': False})
        time.sleep(random.uniform(0.5, 2.0))

sim_failed_login('malicious', '192.122.36.1', count=6)

Event: login_attempt, Role: USER, User: malicious, Time: 2025-05-31 17:08:55.603663
Event: login_attempt, Role: USER, User: malicious, Time: 2025-05-31 17:08:57.573380
Event: login_attempt, Role: USER, User: malicious, Time: 2025-05-31 17:08:58.911744
Event: login_attempt, Role: USER, User: malicious, Time: 2025-05-31 17:08:59.491493
Event: login_attempt, Role: USER, User: malicious, Time: 2025-05-31 17:09:00.943480
Event: login_attempt, Role: USER, User: malicious, Time: 2025-05-31 17:09:02.092104
ALERT: Too many failed login attempts for malicious


In [ ]:
def sim_toggle_spam(user_id, role, user_ip, count=11):
    states = ['on', 'off']
    for i in range(count):
        instrument('toggle_device', role, user_id, user_ip, datetime.now(), {'device_id': 'device_2', 'device_type': 'non-critical', 'state': states[i % 2]})
        time.sleep(random.uniform(0.5, 2.0))

sim_toggle_spam('spammer', 'USER', '192.111.23.6', count=11)

Event: toggle_device, Role: USER, User: spammer, Time: 2025-05-31 17:09:03.955249
Event: toggle_device, Role: USER, User: spammer, Time: 2025-05-31 17:09:05.561186
Event: toggle_device, Role: USER, User: spammer, Time: 2025-05-31 17:09:06.926867
Event: toggle_device, Role: USER, User: spammer, Time: 2025-05-31 17:09:08.607989
Event: toggle_device, Role: USER, User: spammer, Time: 2025-05-31 17:09:09.977817
ALERT: Conflicting toggle states on device_2: off to on
Event: toggle_device, Role: USER, User: spammer, Time: 2025-05-31 17:09:11.728155
ALERT: Activity from previously flagged source: spammer / 192.111.23.6
ALERT: Conflicting toggle states on device_2: on to off
Event: toggle_device, Role: USER, User: spammer, Time: 2025-05-31 17:09:13.398266
ALERT: Activity from previously flagged source: spammer / 192.111.23.6
ALERT: Conflicting toggle states on device_2: off to on
Event: toggle_device, Role: USER, User: spammer, Time: 2025-05-31 17:09:14.580151
ALERT: Activity from previously fl

In [83]:
def sim_power_spike(user_id, user_ip, base=1000, spike=2000):
    for _ in range(6):
        instrument('power_reading', 'USER', user_id, user_ip, datetime.now(), {'value': base})
        time.sleep(random.uniform(0.5, 2.0))
    instrument('power_reading', 'USER', user_id, user_ip, datetime.now(), {'value': spike})

sim_power_spike('spiker', '192.168.2.2', base=1000, spike=2500)

Event: power_reading, Role: USER, User: spiker, Time: 2025-05-31 17:09:19.781609
Event: power_reading, Role: USER, User: spiker, Time: 2025-05-31 17:09:20.748519
Event: power_reading, Role: USER, User: spiker, Time: 2025-05-31 17:09:21.381230
Event: power_reading, Role: USER, User: spiker, Time: 2025-05-31 17:09:23.012151
Event: power_reading, Role: USER, User: spiker, Time: 2025-05-31 17:09:24.868855
Event: power_reading, Role: USER, User: spiker, Time: 2025-05-31 17:09:26.721246
Event: power_reading, Role: USER, User: spiker, Time: 2025-05-31 17:09:28.033678
ALERT: Power spike (2500) exceeds 150% of avg (1187.50)


In [84]:
def sim_admin_behavior(admin_id, admin_ip, hour=10):
    ts = datetime.now().replace(hour=hour, minute=0, second=0)
    states = ['on', 'off']
    for i in range(11):
        instrument('toggle_device', 'ADMIN', admin_id, admin_ip, ts, {'device_id': 'device_3', 'device_type': 'critical', 'state': states[i % 2]})
        time.sleep(random.uniform(0.5, 2.0))
    instrument('power_reading', 'MANAGER', admin_id, admin_ip, ts, {'value': 3000})

sim_admin_behavior('admin_real', '22.15.1.3', hour=14)
sim_admin_behavior('admin_malicious', '155.23.65.1', hour=22)

Event: toggle_device, Role: ADMIN, User: admin_real, Time: 2025-05-31 14:00:00.054422
Event: toggle_device, Role: ADMIN, User: admin_real, Time: 2025-05-31 14:00:00.054422
Event: toggle_device, Role: ADMIN, User: admin_real, Time: 2025-05-31 14:00:00.054422
Event: toggle_device, Role: ADMIN, User: admin_real, Time: 2025-05-31 14:00:00.054422
Event: toggle_device, Role: ADMIN, User: admin_real, Time: 2025-05-31 14:00:00.054422
ALERT: Conflicting toggle states on device_3: off to on
Event: toggle_device, Role: ADMIN, User: admin_real, Time: 2025-05-31 14:00:00.054422
ALERT: Activity from previously flagged source: admin_real / 22.15.1.3
ALERT: Conflicting toggle states on device_3: on to off
Event: toggle_device, Role: ADMIN, User: admin_real, Time: 2025-05-31 14:00:00.054422
ALERT: Activity from previously flagged source: admin_real / 22.15.1.3
ALERT: Conflicting toggle states on device_3: off to on
Event: toggle_device, Role: ADMIN, User: admin_real, Time: 2025-05-31 14:00:00.054422
AL

In [85]:
def sim_flagged_source(user_id, user_ip):
    for _ in range(6):
        instrument('login_attempt', 'USER', user_id, user_ip, datetime.now(), {'success': False})
        time.sleep(random.uniform(0.5, 2.0))
    instrument('login_attempt', 'USER', user_id, user_ip, datetime.now(), {'success': True})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('toggle_device', 'USER', user_id, user_ip, datetime.now(), {'device_id': 'device_4', 'device_type': 'non-critical', 'state': 'on'})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('power_reading', 'USER', user_id, user_ip, datetime.now(), {'value': 950})

sim_flagged_source('flagged_user', '192.168.13.69')

Event: login_attempt, Role: USER, User: flagged_user, Time: 2025-05-31 17:09:55.188769
Event: login_attempt, Role: USER, User: flagged_user, Time: 2025-05-31 17:09:56.600254
Event: login_attempt, Role: USER, User: flagged_user, Time: 2025-05-31 17:09:58.049720
Event: login_attempt, Role: USER, User: flagged_user, Time: 2025-05-31 17:09:58.944161
Event: login_attempt, Role: USER, User: flagged_user, Time: 2025-05-31 17:09:59.960779
Event: login_attempt, Role: USER, User: flagged_user, Time: 2025-05-31 17:10:01.678104
ALERT: Too many failed login attempts for flagged_user
Event: login_attempt, Role: USER, User: flagged_user, Time: 2025-05-31 17:10:02.370432
ALERT: Activity from previously flagged source: flagged_user / 192.168.13.69
Event: toggle_device, Role: USER, User: flagged_user, Time: 2025-05-31 17:10:03.823005
ALERT: Activity from previously flagged source: flagged_user / 192.168.13.69
Event: power_reading, Role: USER, User: flagged_user, Time: 2025-05-31 17:10:05.084148
ALERT: A

In [86]:
def sim_session_hijack(session_id):
    instrument('session_start', 'USER', 'user_hijack', '192.168.1.10', datetime.now(), {'session_id': session_id, 'session_type': 'interactive'})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('session_start', 'USER', 'user_hijack', '203.0.113.55', datetime.now(), {'session_id': session_id, 'session_type': 'interactive'})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('session_end', 'USER', 'user_hijack', '203.0.113.55', datetime.now(), {'session_id': session_id})

sim_session_hijack('sesh_hijack')

Event: session_start, Role: USER, User: user_hijack, Time: 2025-05-31 17:10:05.103153
Event: session_start, Role: USER, User: user_hijack, Time: 2025-05-31 17:10:05.776167
ALERT: Session hijacking suspected for session 'sesh_hijack' used from multiple IPs: {'203.0.113.55', '192.168.1.10'}
Event: session_end, Role: USER, User: user_hijack, Time: 2025-05-31 17:10:07.774682


In [87]:
def sim_location_anomaly(user_id, ip_list):
    for ip in ip_list:
        instrument('login_attempt', 'USER', user_id, ip, datetime.now(), {'success': True})
        time.sleep(random.uniform(0.5, 2.0))

sim_location_anomaly('geo_user', ['192.168.1.1', '203.0.113.1', '172.16.1.1'])

Event: login_attempt, Role: USER, User: geo_user, Time: 2025-05-31 17:10:07.792429
Event: login_attempt, Role: USER, User: geo_user, Time: 2025-05-31 17:10:09.752558
Event: login_attempt, Role: USER, User: geo_user, Time: 2025-05-31 17:10:11.408157
ALERT: Unrealistic login locations for geo_user: US to UK in 0.00 hours
ALERT: Unrealistic login locations for geo_user: UK to CN in 0.00 hours


In [88]:
def sim_device_overload(device_id, user_ip_pairs):
    states = ['on', 'off']
    for i, (user_id, ip) in enumerate(user_ip_pairs):
        role = 'ADMIN' if user_id == 'user1' else 'USER'
        instrument('toggle_device', role, user_id, ip, datetime.now(), {'device_id': device_id, 'device_type': 'critical', 'state': states[i % 2]})
        time.sleep(random.uniform(0.5, 2.0))

sim_device_overload('device_5', [('user1', '192.168.5.1'), ('user2', '192.168.5.2'), ('user3', '192.168.5.3'), ('user4', '192.168.5.4')])

Event: toggle_device, Role: ADMIN, User: user1, Time: 2025-05-31 17:10:12.309714
Event: toggle_device, Role: USER, User: user2, Time: 2025-05-31 17:10:13.764806
Event: toggle_device, Role: USER, User: user3, Time: 2025-05-31 17:10:15.717407
ALERT: Device overload on device_5 (critical) by users: {'user3', 'user1', 'user2'}
Event: toggle_device, Role: USER, User: user4, Time: 2025-05-31 17:10:16.258872
ALERT: Device overload on device_5 (critical) by users: {'user3', 'user1', 'user4', 'user2'}


In [ ]:
def sim_long_session(user_id, user_ip, session_id):
    start_time = datetime.now() - timedelta(hours=25)
    instrument('session_start', 'USER', user_id, user_ip, start_time, {'session_id': session_id, 'session_type': 'interactive'})
    time.sleep(random.uniform(0.5, 2.0))
    
    # Simulate activity 13 hours ago
    active_time = datetime.now() - timedelta(hours=13)
    instrument('power_reading', 'USER', user_id, user_ip, active_time, {'value': 1000, 'session_id': session_id})
    time.sleep(random.uniform(0.5, 2.0))
    instrument('session_end', 'USER', user_id, user_ip, datetime.now(), {'session_id': session_id})

sim_long_session('long_user', '192.168.6.1', 'long_session')

Event: session_start, Role: USER, User: long_user, Time: 2025-05-30 16:10:17.079419
Event: power_reading, Role: USER, User: long_user, Time: 2025-05-31 04:10:18.511136
Event: session_end, Role: USER, User: long_user, Time: 2025-05-31 17:10:19.103323


In [90]:
visualise_data()
print('Failed Logins:', failed_logins)
print('Toggle Events:', toggle_events)
print('Power History:', [v for _, v in power_history])
print('Session Creations:', session_creations)
print('Device Access Log:', device_access_log)
print('Flagged Users:', flagged_users)
print('Flagged IPs:', flagged_ips)
print('Login Locations:', login_locations)
print('Session Durations:', session_durations)

Failed Logins: {'malicious': [datetime.datetime(2025, 5, 31, 17, 8, 55, 603663), datetime.datetime(2025, 5, 31, 17, 8, 57, 573380), datetime.datetime(2025, 5, 31, 17, 8, 58, 911744), datetime.datetime(2025, 5, 31, 17, 8, 59, 491493), datetime.datetime(2025, 5, 31, 17, 9, 0, 943480), datetime.datetime(2025, 5, 31, 17, 9, 2, 92104)], 'flagged_user': [datetime.datetime(2025, 5, 31, 17, 9, 55, 188769), datetime.datetime(2025, 5, 31, 17, 9, 56, 600254), datetime.datetime(2025, 5, 31, 17, 9, 58, 49720), datetime.datetime(2025, 5, 31, 17, 9, 58, 944161), datetime.datetime(2025, 5, 31, 17, 9, 59, 960779), datetime.datetime(2025, 5, 31, 17, 10, 1, 678104)]}
Toggle Events: {'normal_user': [datetime.datetime(2025, 5, 31, 17, 8, 51, 996071)], 'spammer': [datetime.datetime(2025, 5, 31, 17, 9, 3, 955249), datetime.datetime(2025, 5, 31, 17, 9, 5, 561186), datetime.datetime(2025, 5, 31, 17, 9, 6, 926867), datetime.datetime(2025, 5, 31, 17, 9, 8, 607989), datetime.datetime(2025, 5, 31, 17, 9, 9, 977817